<a href="https://colab.research.google.com/github/ekrombouts/gcai_zuster_fietje/blob/main/300_GenCareAICarePlanGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Creating a Careplan Dataset from GenCareAI Client records

**Author:** Eva Rombouts  
**Date:** 2024-09-16  
**Updated:** 2024-11-30

### Description
This notebook summarizes nursing home client notes into the Careplan format using OpenAI’s GPT model via LangChain. It processes the data, generates careplans, and prepares the dataset by splitting it into training, validation, and test sets, and uploads it to the Hugging Face Hub for use in machine learning models.

## Environment Setup and Library Imports

In [1]:
# When in Colab
from google.colab import drive, userdata
import os


drive.mount('/content/drive')
base_dir = "/content/drive/My Drive/Colab Notebooks/GenCareAI"
open_ai_api_key = userdata.get("GCI_OPENAI_API_KEY")

!pip install -q datasets langchain langchain_community langchain_openai

verbose = True

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import random
import pandas as pd
from tqdm import tqdm  # Progress bar
from datasets import load_dataset, Dataset, DatasetDict  # For loading and managing datasets
from typing import List

# Langchain modules
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.callbacks import get_openai_callback

# Pydantic library for data validation
from pydantic import BaseModel, Field

# OpenAI API integration using langchain
from langchain_openai import ChatOpenAI

# Scikit-learn library for splitting datasets into training and testing sets
from sklearn.model_selection import train_test_split


In [3]:
# Set parameters
seed = 6

# Data paths
nursing_care_home_name = "Gardenia"
# For reading data
path_hf_records = f"ekrombouts/{nursing_care_home_name}_records"
path_hf_clients = f"ekrombouts/{nursing_care_home_name}_clients"

# For writing data
path_hf_careplan = f"ekrombouts/{nursing_care_home_name}_Careplan_dataset"
commit_message = "Careplan dataset"

# File path for saving the generated Careplans
fn_responses = os.path.join(base_dir, f"data/care_pal/{nursing_care_home_name}_Careplan_dataset.pkl")

# Settings for Careplan generation
model = "gpt-4o-mini-2024-07-18"
temperature = 0.3

sep_line = 50 * '-'


## Loading and Preprocessing Data
Client records and notes from fictional clients of a nursing home are loaded, cleaned, and processed. The notes are grouped by week

In [4]:
# Load dataset from Hugging Face and preprocess
dataset = load_dataset(path_hf_records)
df_records = dataset['train'].to_pandas()

# Floor datetime to the first day of the month
df_records['week'] = df_records['datetime'].dt.to_period('W').dt.to_timestamp()

# Group records by 'client_id' and 'week', concatenating notes into one string
df = (df_records
    .dropna()
    .assign(week=lambda df: pd.to_datetime(df['datetime']).dt.to_period('W').dt.to_timestamp()) # Add 'week' column
    .groupby(['client_id', 'week'])
    .agg({'note': lambda x: '\n'.join(x)}) # Concatenate 'note' values
    .reset_index()
    .rename(columns={'note': 'weeknotes'})
)

if verbose:
  print(f"Rows in original df: {df_records.shape[0]}, rows in processed df: {df.shape[0]}\n")
  print(f"SAMPLES{sep_line}\n{df.sample(3)}\n")
  print(f"\nContext column (weeknotes) example:{sep_line}\n{df['weeknotes'].iloc[0]}")


Rows in original df: 28478, rows in processed df: 1681

SAMPLES--------------------------------------------------
     client_id       week                                          weeknotes
1523    mag017 2022-12-19  Client vandaag wat stiller en teruggetrokken. ...
19      cro001 2022-08-15  Client was vanochtend rustig en vredig in zijn...
868     iri010 2022-04-11  Mevrouw was vanmorgen onrustig en had probleme...


Context column (weeknotes) example:--------------------------------------------------
De cliënt is vandaag gearriveerd in het verpleeghuis. Hij is voorgesteld aan het personeel en de medebewoners. Het zorgplan is opgesteld en de benodigde hulp bij persoonlijke verzorging is gepland.
Cliënt heeft hulp nodig gehad bij aankleden en eten. Hij kon korte afstanden lopen met behulp van een rollator.
Cliënt leek wat verward en vergeetachtig in de avond. Extra observatie gedaan, gedrag goed in de gaten gehouden.
Vandaag had de cliënt wat moeite met de lichamelijke verzorging. Ru

## LLM Response Generation: Careplans

In [5]:
class CarePlanItem(BaseModel):
    problem: str = Field(..., description="Beschrijving van het zorgprobleem. Zorg dat er slechts één probleem wordt beschreven")
    care_goal: str = Field(..., description="Beschrijving van het zorgdoel")
    interventions: List[str] = Field(..., description="Beschrijving van 1 tot max 3 interventies")

class CarePlan(BaseModel):
    careplan: List[CarePlanItem] = Field(..., description="Lijst van 1 tot max 3 zorgdoelen en interventies")

# Set up a parser to handle the output and inject instructions into the prompt template
pyd_parser = PydanticOutputParser(pydantic_object=CarePlan)

In [6]:
template="""
Schrijf een zorgplan op basis van onderstaande rapportages.

Belangrijk:
- Gebruik uitsluitend informatie uit de rapportages. Voeg geen eigen interpretaties toe.
- Richt je op algemene observaties en patronen, zonder de details van de rapportages over te nemen.

---
RAPPORTAGES:
{rapportages}
---

{format_instructions}
"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["rapportages"],
    partial_variables={"format_instructions": pyd_parser.get_format_instructions()},
)


In [7]:

llm = ChatOpenAI(
    api_key=open_ai_api_key,
    model=model,
    temperature=temperature
)

chain = prompt_template | llm | pyd_parser


if verbose:
    sample_id = 5
    sample_context = df['weeknotes'].iloc[sample_id]

    sample_prompt = template.format(
            rapportages=sample_context,
            format_instructions=pyd_parser.get_format_instructions()
    )

    result = chain.invoke({"rapportages": sample_context})

    # print the CarePlan
    print(sample_prompt)

    print("RESPONSE")

    for i, item in enumerate(result.careplan):
        print(f"Probleem {i+1}:") #Added problem number
        print(item.problem)
        print(item.care_goal)
        for intervention in item.interventions:
            print(f"- {intervention}")


Schrijf een zorgplan op basis van onderstaande rapportages.

Belangrijk:
- Gebruik uitsluitend informatie uit de rapportages. Voeg geen eigen interpretaties toe.
- Richt je op algemene observaties en patronen, zonder de details van de rapportages over te nemen.

---
RAPPORTAGES:
Diabetescontrole vandaag positief, bloedsuikerwaarden stabiel. Afspraken hierover doorgenomen met cliënt om zelfstandigheid te stimuleren.
Familiebezoek liep vlot en harmonieus, cliënt leek zich op zijn gemak te voelen. Samen fotoalbums bekeken.
Cliënt had vanavond last van hallucinaties, zag schaduwen op de muur en raakte in paniek. Zachtjes afgeleid en kalmerend toegesproken.
Dhr. had vandaag moeite met opstaan en stond wankel op zijn benen. Samen naar de fysiotherapeut geweest voor oefeningen.
Er was vanmiddag sprake van toenemende verwardheid en onrust, cliënt roept vaak om zijn moeder. Troostend bijgestaan en afleiding geboden.
Cliënt reageerde agressief tegen medebewoners tijdens het avondeten. Direct in

In [8]:
# Function to generate the Careplan
def generate_careplan(notes: str) -> CarePlan:
    try:
        result = chain.invoke({"rapportages": notes})
        return result
    except Exception as e:
        print(f"Error generating Careplan: {e}")
        return None

# Load the previously saved dataframe if it exists, otherwise start fresh
if os.path.exists(fn_responses):
    df = pd.read_pickle(fn_responses)
else:
    df['careplan_response'] = None  # Ensure the column exists

# Create a callback instance to track cost
with get_openai_callback() as cb:

    # Generate Summaries, process only new entries
    with tqdm(total=len(df), desc="Generating Careplans") as pbar:  # Set the total for the progress bar
        for idx, row in df.iterrows():
            if pd.isna(df.at[idx, 'careplan_response']):  # Process only new rows or missing responses
                careplan = generate_careplan(row['weeknotes'])
                df.at[idx, 'careplan_response'] = careplan

            # Update the progress bar
            pbar.update(1)

            # Save progress every 100 iterations
            if idx % 100 == 0:
                df.to_pickle(fn_responses)
                print(f"Checkpoint saved at index {idx}, total cost so far: ${cb.total_cost:.4f}")

    # Save the final result
    df.to_pickle(fn_responses)
    print("Processing complete and final dataframe saved.")
    print(f"Total cost: ${cb.total_cost:.4f}")


Generating Careplans:   6%|▌         | 102/1681 [00:00<00:02, 661.97it/s]

Checkpoint saved at index 0, total cost so far: $0.0000
Checkpoint saved at index 100, total cost so far: $0.0000


Generating Careplans:  12%|█▏        | 202/1681 [00:00<00:03, 478.26it/s]

Checkpoint saved at index 200, total cost so far: $0.0000
Checkpoint saved at index 300, total cost so far: $0.0000


Generating Careplans:  36%|███▌      | 602/1681 [00:00<00:01, 869.59it/s]

...

Generating Careplans:  85%|████████▌ | 1432/1681 [00:01<00:00, 919.97it/s]

Checkpoint saved at index 1300, total cost so far: $0.0000
Checkpoint saved at index 1400, total cost so far: $0.0000


Generating Careplans: 100%|██████████| 1681/1681 [00:01<00:00, 890.14it/s]

Checkpoint saved at index 1500, total cost so far: $0.0000
Checkpoint saved at index 1600, total cost so far: $0.0000


Processing complete and final dataframe saved.
Total cost: $0.0000


## Dataset Creation, Splitting and Saving

In [9]:
instruction = '''Schrijf een zorgplan op basis van onderstaande rapportages. Gebruik alleen informatie uit de rapportages, zonder eigen interpretaties toe te voegen.

Formatteer de output als een JSON-instantie die voldoet aan het onderstaande JSON-schema.
```
{”$defs”:{“CarePlanItem”:{“properties”:{“problem”:{“title”:“Problem”,“type”:“string”},“care_goal”:{“title”:“Care Goal”,“type”:“string”},“interventions”:{“title”:“Interventions”,“type”:“array”,“items”:{“type”:“string”}}},“required”:[“problem”,“care_goal”,“interventions”],“title”:“CarePlanItem”,“type”:“object”}},“properties”:{“careplan”:{“title”:“Careplan”,“type”:“array”,“items”:{”$ref”:”#/$defs/CarePlanItem”}}},“required”:[“careplan”]}
```
'''

In [10]:
# use method chaining to rename columns and reorder
df_careplan = (
    df.rename(columns={'weeknotes': 'context', 'careplan_response': 'response'})
    .assign(
        response=lambda df: df['response'].astype(str),
        instruction=instruction
    )
    [['client_id', 'week', 'context', 'instruction', 'response']]
)

In [11]:
# Split the dataset and push to Hugging Face hub

# Convert df to Hugging Face dataset
dataset = Dataset.from_pandas(
    df=df_careplan,
    preserve_index=False
)

# Split the dataset into training(80%), validation(10%), and test(10%) sets
train_testvalid_split = dataset.train_test_split(
    test_size=0.2,
    seed=seed
)
test_valid_split = train_testvalid_split['test'].train_test_split(
    test_size=0.5,
    seed=seed
)

dataset_dict = DatasetDict({
    'train': train_testvalid_split['train'],
    'validation': test_valid_split['train'],
    'test': test_valid_split['test'],
})

# # Push the dataset to Hugging Face Hub
# dataset_dict.push_to_hub(path_hf_careplan,
#                          commit_message=commit_message,
#                          private=True)



In [13]:
dataset_dict ['test'][6]

{'client_id': 'iri004',
 'week': Timestamp('2023-01-02 00:00:00'),
 'context': 'Bij de ochtendzorg viel op dat cliënte moeite had met eten vanwege de nierproblemen. Langzamer gegeten en voldoende vocht aangeboden.\nActief deelgenomen aan het creatieve knutselproject. Cliënte was geconcentreerd bezig en leek te genieten van de bezigheid.\nTijdens het avondeten leek cliënte vermoeider dan normaal. Extra rustmoment ingelast en geholpen met het naar bed gaan.\nClient heeft moeite met eten vanwege slecht zicht. Leesbril ingezet tijdens ontbijt, dit leek te helpen.\nClient vrolijk en actief tijdens de verzorging. Zong zelfs een liedje tijdens het wassen en aankleden.\nClient heeft goed gereageerd op het krijgen van haar rollator. Kon zelfstandig kleine stukjes lopen in de gang.\nClient weigerde in eerste instantie haar medicijnen in te nemen. Met wat geduld en uitleg toch gelukt om haar ze te laten nemen.\nClient had wat moeite met herinneren waar ze was tijdens de lunch. Rustig uitgelegd en

In [ ]:
commit_message